In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import logging

# Setup paths
PROJECT_ROOT = os.path.dirname(os.path.dirname(os.getcwd()))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Setup directories
RESULTS_DIR = os.path.join(os.getcwd(), 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)

# Imports
from preprocessing.data_container import DataContainer
from utils.evaluation import cindex_score
from models.modelling_process import ModellingProcess
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from utils.evaluation import EarlyStoppingMonitor

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [11]:
import pickle

with open('../../results/pipe/test.pkl', 'rb') as f:
    data = pickle.load(f)

## Intersection and imputed data 

### 1.1 Model with only exprs. data; Intersection + imputed

In [11]:
# set early stopping monitor 
monitor = EarlyStoppingMonitor(10, 5)

# Data configuration
DATA_CONFIG = {
    'use_pca': False,
    'pca_threshold': 0.85,
    'gene_type': 'intersection',
    'use_imputed': True,
    'select_random' : False, 
    'use_cohorts': False, 
    'clinical_covs' : ['TISSUE', 'AGE','GLEASON_SCORE']
}

# Model configuration
# MODEL_CONFIG = {
#     'params_cv': {
#         'model__n_estimators': [500],
#         'model__learning_rate': [0.1],
#         'model__max_depth': [3, 5],
#         'model__min_samples_split': [5, 10],
#         'model__min_samples_leaf': [3, 5],
#         'model__subsample': [0.9],
#         'model__max_features': ['sqrt', None]
#     },
#     'refit': True, 
#     'do_nested_resampling': False, 
#     'monitor' : monitor, 
#     'path' : RESULTS_DIR, 
#     'fname_cv' : 'results_PCA'}

# Model configuration
MODEL_CONFIG = {
    'params_cv': {
        'model__n_estimators': [200],
        'model__learning_rate': [0.1],
        #'model__max_depth': [3, 5],
        'model__min_samples_split': [10],
        #'model__min_samples_leaf': [3, 5],
        'model__subsample': [0.8],
        'model__max_features': ['log2'], 
        'model__n_iter_no_change' : [10], 
        'model__validation_fraction' : [0.1]
    },
    'refit': True, 
    'do_nested_resampling': True, 
    #'monitor' : monitor, 
    'path' : RESULTS_DIR, 
    'fname_cv' : 'results_PCA'}

# validation_fraction=0.1 as a mean to inclued early stopping
gb_pipeline_steps = [('model', GradientBoostingSurvivalAnalysis())]


In [3]:
mp = ModellingProcess()

In [7]:
mp.prepare_data(DATA_CONFIG, PROJECT_ROOT)

2024-12-29 10:23:40,746 - INFO - Loading data...
2024-12-29 10:24:10,202 - INFO - Found clinical data specification
2024-12-29 10:24:10,282 - INFO - Loaded data: 1091 samples, 13219 features


<class 'pandas.core.frame.DataFrame'>
Index: 1091 entries, Atlanta_2014_Long.PT081 to Stockholm_2016_Ross_Adams.STKHLM9246
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AGE                1091 non-null   float64
 1   TISSUE             1091 non-null   object 
 2   CLIN_T_STAGE       1091 non-null   object 
 3   PATH_T_STAGE       1091 non-null   object 
 4   GLEASON_SCORE      1091 non-null   int64  
 5   PRE_OPERATIVE_PSA  1091 non-null   float64
 6   MONTH_TO_BCR       1091 non-null   float64
 7   BCR_STATUS         1091 non-null   int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 76.7+ KB
None


In [9]:
mp.X

,TISSUE_FFPE,TISSUE_Fresh_frozen,TISSUE_Snap_frozen,AGE,GLEASON_SCORE,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,...,ENSG00000277972,ENSG00000278053,ENSG00000278195,ENSG00000278259,ENSG00000278311,ENSG00000278318,ENSG00000278505,ENSG00000278535,ENSG00000278540,ENSG00000282608
Atlanta_2014_Long.PT081,1.0,0,0,78.0,7,-3.006631,-0.622789,1.705887,0.234710,0.906281,...,-0.100000,1.874472,0.412771,-0.201057,-0.023212,1.346444,-0.100000,-0.459605,1.673088,0.236356
Atlanta_2014_Long.PT127,1.0,0,0,61.0,6,0.044560,-0.622789,0.520111,0.840715,0.440554,...,-0.100000,0.464558,1.473907,-0.201057,-1.225928,0.861876,-0.100000,-0.459605,1.066850,1.450024
Atlanta_2014_Long.PT168,1.0,0,0,63.0,6,0.975390,-0.622789,0.104950,1.722182,-0.308974,...,-0.100000,0.297391,-0.500212,-0.201057,-1.225928,0.183742,-0.100000,-0.459605,1.766876,-1.059580
Atlanta_2014_Long.PT184,1.0,0,0,58.0,7,0.744177,-0.622789,0.137887,0.569978,0.234893,...,-0.100000,-0.102693,1.919342,-0.201057,-1.225928,0.086797,-0.100000,-0.459605,-0.051815,0.795535
Atlanta_2014_Long.PT199,1.0,0,0,57.0,7,0.316196,-0.622789,0.595612,0.282694,-0.553280,...,-0.100000,1.265752,-0.500212,-0.201057,1.013715,-0.187605,-0.100000,-0.459605,0.601972,-1.059580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stockholm_2016_Ross_Adams.STKHLM8462,0,1.0,0,63.0,7,1.054221,-0.221158,0.914313,1.166952,-0.987476,...,1.259790,-0.785377,-0.393627,-0.087867,0.629437,1.177046,-1.055679,0.131266,-0.901919,-0.153249
Stockholm_2016_Ross_Adams.STKHLM8659,0,1.0,0,63.0,7,-0.376987,-0.595446,-0.413672,0.974346,0.512080,...,-0.770326,-1.195006,-2.131437,-0.020858,-0.426975,0.448978,0.720650,-0.539557,0.044055,-0.225257
Stockholm_2016_Ross_Adams.STKHLM9157,0,1.0,0,63.0,7,-0.096840,0.003729,-0.922901,-1.304991,0.077976,...,0.838754,-0.007334,-0.356743,0.636358,-0.564288,1.464345,-2.986711,0.773578,0.778639,-0.771943
Stockholm_2016_Ross_Adams.STKHLM9161,0,1.0,0,63.0,7,-0.084071,-0.351288,-0.234087,-0.616688,-0.300022,...,0.816947,-0.640533,-0.966573,1.049117,-0.522911,0.139349,-1.012419,0.657811,1.829945,-0.561905


In [19]:
len(data.predict(mp.X))

1091

In [12]:
nstd_res_result = mp.do_modelling(gb_pipeline_steps, MODEL_CONFIG)

2024-12-29 10:26:12,155 - INFO - No additional monitoring detected
2024-12-29 10:26:12,156 - INFO - Start model training...
2024-12-29 10:26:12,157 - INFO - Input data shape: X=(1091, 13219)
2024-12-29 10:26:12,158 - INFO - Nested resampling...
2024-12-29 10:26:12,159 - INFO - Starting nested resampling...
2024-12-29 10:26:12,161 - INFO - Data shape: X=(1091, 13219), groups=9 unique
2024-12-29 10:26:12,165 - INFO - 
Outer fold 1
2024-12-29 10:26:12,257 - INFO - Test cohort: Atlanta_2014_Long


Fitting 8 folds for each of 1 candidates, totalling 8 fits


c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
2024-12-29 10:26:26,335 - INFO - Best parameters: {'model__learning_rate': 0.1, 'model__max_features': 'log2', 'model__min_samples_split': 10, 'model__n_estimators': 200, 'model__n_iter_no_change': 10, 'model__subsample': 0.8, 'model__validation_fraction': 0.1}
2024-12-29 10:26:26,336 - INFO - Test score: 0.676
2024-12-29 10:26:26,337 - INFO - 
Outer fold 2
2024-12-29 10:26:26,426 - INFO - Test cohort: Belfast_2018_Jain


Fitting 8 folds for each of 1 candidates, totalling 8 fits


c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
2024-12-29 10:26:37,100 - INFO - Best parameters: {'model__learning_rate': 0.1, 'model__max_features': 'log2', 'model__min_samples_split': 10, 'model__n_estimators': 200, 'model__n_iter_no_change': 10, 'model__subsample': 0.8, 'model__validation_fraction': 0.1}
2024-12-29 10:26:37,102 - INFO - Test score: 0.622
2024-12-29 10:26:37,104 - INFO - 
Outer fold 3
2024-12-29 10:26:37,202 - INFO - Test cohort: CPC_GENE_2017_Fraser


...

c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
2024-12-29 10:27:46,129 - INFO - Best parameters: {'model__learning_rate': 0.1, 'model__max_features': 'log2', 'model__min_samples_split': 10, 'model__n_estimators': 200, 'model__n_iter_no_change': 10, 'model__subsample': 0.8, 'model__validation_fraction': 0.1}
2024-12-29 10:27:46,129 - INFO - Test score: 0.692
2024-12-29 10:27:46,139 - INFO - 
Outer fold 9
2024-12-29 10:27:46,223 - INFO - Test cohort: Stockholm_2016_Ross_Adams


Fitting 8 folds for each of 1 candidates, totalling 8 fits


c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
2024-12-29 10:27:56,501 - INFO - Best parameters: {'model__learning_rate': 0.1, 'model__max_features': 'log2', 'model__min_samples_split': 10, 'model__n_estimators': 200, 'model__n_iter_no_change': 10, 'model__subsample': 0.8, 'model__validation_fraction': 0.1}
2024-12-29 10:27:56,511 - INFO - Test score: 0.713
2024-12-29 10:27:56,511 - INFO - Aggregated results:
2024-12-29 10:27:56,511 - INFO - Mean score: 0.708 ± 0.064
2024-12-29 10:27:56,511 - INFO - Individual scores: [np.float64(0.6763005780346821), np.float64(0.6221304873137334), np.float64(0.6538461538461539), np.float64(0.682

Fitting 9 folds for each of 1 candidates, totalling 9 fits


c:\Users\laeti\PCaPrognostics\.venv\Lib\site-packages\sklearn\utils\validation.py:877: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
2024-12-29 10:28:12,286 - INFO - Saved model to c:\Users\laeti\PCaPrognostics\models\grad_boost\results\model
2024-12-29 10:28:12,286 - WARNING - Won't save any cv results, since its not provided
2024-12-29 10:28:12,301 - INFO - Saved pipe to c:\Users\laeti\PCaPrognostics\models\grad_boost\results\pipe


In [ ]:
mp.save_results(RESULTS_DIR, 'gb_intersect_imp', model = mp.cmplt_model, cv_results = mp.resampling_cmplt, pipe = mp.cmplt_pipeline)

In [24]:
df_features = pd.DataFrame()
df_features['feat'] = mp.X.columns
df_features['imps'] = mp.cmplt_model.feature_importances_

df_features.sort_values(by = 'imps', ascending=False)[df_features['imps'] > 0]

C:\Users\laeti\AppData\Local\Temp\ipykernel_2220\3881194196.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_features.sort_values(by = 'imps', ascending=False)[df_features['imps'] > 0]


,feat,imps
2387,ENSG00000104313,0.006392
4361,ENSG00000123485,0.005313
5208,ENSG00000132122,0.004998
3186,ENSG00000111670,0.004415
2639,ENSG00000106025,0.003916
...,...,...
4564,ENSG00000125508,0.000064
9371,ENSG00000167487,0.000059
1493,ENSG00000088543,0.000050
8409,ENSG00000162620,0.000028


### 1.2 Model with only pData

In [ ]:
# set early stopping monitor 
monitor = EarlyStoppingMonitor(10, 5)

# Data configuration
DATA_CONFIG = {
    'use_pca': False,
    'pca_threshold': 0.85,
    'gene_type': 'intersection',
    'use_imputed': True,
    'select_random' : False, 
    'use_cohorts': False,
    'only_pData' : True
}

# Model configuration
MODEL_CONFIG = {
    'params_cv': {
        'model__n_estimators': [500],
        'model__learning_rate': [0.1],
        'model__max_depth': [3, 5],
        'model__min_samples_split': [5, 10],
        'model__min_samples_leaf': [3, 5],
        'model__subsample': [0.9],
        'model__max_features': ['sqrt', None]
    },
    'refit': True, 
    'do_nested_resampling': True, 
    'monitor' : monitor, 
    'path' : RESULTS_DIR, 
    'fname_cv' : 'results_PCA'}

# validation_fraction=0.1 as a mean to inclued early stopping
gb_pipeline_steps = [('model', GradientBoostingSurvivalAnalysis())]


### 1.3 Model with both exprs. and pData